In [1]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep
import json,os,re

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--incognito')
options.add_argument('--disable-popup-blocking')

driver = webdriver.Chrome(options = options)

url = "https://www.ibon.com.tw/retail_inquiry.aspx#gsc.tab=0"
cityList = []
listData = []

def visit():
    driver.get(url)

def getCity():    
    for city in driver.find_elements(By.CSS_SELECTOR,"select#Class1 option"):
        cityList.append(city.get_attribute('value'))

def getStore():
    for i in range(2): #range(len(cityList)): 抓所有全台門市，range(2): 抓台北市與新北市
        city = "select#Class1 option[value='"+cityList[i]+"']"
        driver.find_element(By.CSS_SELECTOR, city).click()
        try:
            WebDriverWait(driver,20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR,"table.font16 tr"))
            )
            stores = driver.find_elements(By.CSS_SELECTOR, "table.font16 tr")
            sleep(10)
        except TimeoutException:
            print("連線逾時, 即將關閉瀏覽器")
            sleep(3)
            driver.quit()
        
        for store in stores:
            store_id = store.find_elements(By.CSS_SELECTOR, "td")[0].get_attribute("innerText")
            name = store.find_elements(By.CSS_SELECTOR, "td")[1].get_attribute("innerText")
            address = store.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("innerText")
            listData.append({
                'area': cityList[i][2:],
                'id': store_id,
                'name': name,
                'address': address
            })
def saveJson():
    file = open("711_stores.json", "w", encoding="utf-8")
    file.write(json.dumps(listData, ensure_ascii=False))
    file.close()           

def close():
    driver.quit()
    
if __name__ == "__main__":
    visit()
    getCity()
    getStore()
    saveJson()
    close()

In [2]:
# 轉成csv檔
import json
import csv
totalData = []
fp = open("711_stores.json", "r", encoding="utf-8")
strJson = fp.read()
fp.close()
listResult = json.loads(strJson)

headers = ["area", "id", "name", "address"]
with open('711_stores.csv','w', encoding='utf-8', newline='') as f:
    write_csv = csv.DictWriter(f , headers)
    write_csv.writeheader()
    write_csv.writerows(listResult)

In [9]:
import pandas as pd

rental = pd.read_csv("711_stores.csv")

rental = rental.drop(rental.index[0])
rental.to_csv('711_stores_1.csv', header = True, index = False)